In [1]:
%cd /Users/tomtalpir/Random/pyERGM

/Users/tomtalpir/Random/pyERGM


In [2]:
from utils import *
from ergm import ERGM

import pandas as pd
import numpy as np

### Initialize an ERGM with random coefficients

In [3]:
n_nodes = 10
stats_calculator = NetworkStatistics(metric_names=["num_edges"], )


stats = [NumEdges(), CustomMetric(lambda x: ), ]
ergm = ERGM(n_nodes, stats_calculator, is_directed=False)
ergm = ERGM(n_nodes, stats_calculator, is_directed=True)

ergm.print_model_parameters()

Number of nodes: 10
Thetas: [-0.29766759]
Normalization factor approx: 50.779457576278446
Is directed: False


### Sample networks from the distribution, and calculate some statistics on the distribution

In [4]:
network_stats = []
for i in range(10):
    W = ergm.sample_network(sampling_method="NaiveMetropolisHastings", steps=1000)
    G = connectivity_matrix_to_G(W, directed=False)
    edge_count = len(G.edges())
    triangle_count = sum(nx.triangles(G).values()) // 3

    network_stats.append({"edge_count": edge_count, "triangle_count": triangle_count})

network_stats_df = pd.DataFrame(network_stats)
network_stats_df.head(20)


,edge_count,triangle_count
0,33,47
1,35,57
2,36,62
3,36,57
4,39,79
5,36,63
6,34,53
7,38,76
8,38,72
9,32,46


## Fit an ERGM

We begin by initializing an ERGM with predefined parameters - 

In [42]:
n_nodes = 5
stats_calculator = NetworkStatistics(metric_names=["num_edges"])

theta = -np.log(3)
ergm = ERGM(n_nodes, stats_calculator, is_directed=False, initial_thetas=[theta])

print("Baseline ERGM parameters - ")
ergm.print_model_parameters()


Baseline ERGM parameters - 
Number of nodes: 5
Thetas: [-1.0986122886681098]
Normalization factor approx: 19.45775812919028
Is directed: False


We now sample from this model a network that will become our "observed network", when estimating our fit performance.

In [43]:
W = ergm.sample_network(sampling_method="NaiveMetropolisHastings", steps=1000)

G = connectivity_matrix_to_G(W, directed=False)

real_edge_count = len(G.edges())
real_triangle_count = sum(nx.triangles(G).values()) // 3

print(f"Sampled a random network from a model with theta = {theta}")
print(W)
print(f"Network has statistics - edge count: {real_edge_count}, triangle count: {real_triangle_count}")

Sampled a random network from a model with theta = -1.0986122886681098
[[0. 0. 0. 1. 1.]
 [0. 0. 1. 1. 1.]
 [0. 1. 0. 0. 1.]
 [1. 1. 0. 0. 1.]
 [1. 1. 1. 1. 0.]]
Network has statistics - edge count: 7, triangle count: 3


Now, fit a random ERGM to create networks with similar statistics

In [46]:
fitted_ergm = ERGM(n_nodes, stats_calculator, is_directed=False)
print(f"Initial ERGM parameters:")
fitted_ergm.print_model_parameters()

pre_fit_W_matrices = []
samples_before_fit = []

n_samples = 10

for i in range(n_samples):
    sampled_W = fitted_ergm.sample_network(sampling_method="NaiveMetropolisHastings", steps=1000)
    G = connectivity_matrix_to_G(sampled_W, directed=False)
    edge_count = len(G.edges())
    triangle_count = sum(nx.triangles(G).values()) // 3

    samples_before_fit.append({"edge_count": edge_count, "triangle_count": triangle_count})
    pre_fit_W_matrices.append(sampled_W)

samples_before_fit_df = pd.DataFrame(samples_before_fit)
print("")
# Print mean for each metric - 
print(f"Mean of samples before fitting, calculated on {n_samples} samples- ")
print(samples_before_fit_df.mean())


Initial ERGM parameters:
Number of nodes: 5
Thetas: [-0.33698419]
Normalization factor approx: 43.1253267687915
Is directed: False

Mean of samples before fitting, calculated on 10 samples- 
edge_count        8.0
triangle_count    5.0
dtype: float64


We now fit the ERGM!

In [47]:

print(f"Fitting ERGM...")
fitted_ergm.fit(W, optimization_options={'disp': True, 'maxiter': 500, 'maxfev': 500}, n_networks_for_norm=500, n_mcmc_steps=100)
print("Done fitting!")


Fitting ERGM...
	Starting fit with initial normalization factor: 43.1253267687915
	Optimization options: {'disp': True, 'maxiter': 500, 'maxfev': 500}
	Optimization result:
	Theta: [-0.15381744]
	Normalization factor: 135.44816303424523
       message: Maximum number of function evaluations has been exceeded.
       success: False
        status: 1
           fun: 5.963890498779282
             x: [-1.538e-01]
           nit: 186
          nfev: 500
 final_simplex: (array([[-1.538e-01],
                       [-1.538e-01]]), array([ 5.964e+00,  5.984e+00]))
Done fitting!


/Users/tomtalpir/Random/pyERGM/ergm.py:130: RuntimeWarning: Maximum number of function evaluations has been exceeded.
  result = minimize(negative_log_likelihood, self._thetas, method='Nelder-Mead', options=optimization_options)


In [48]:
print(f"Post fit ERGM parameters:")
fitted_ergm.print_model_parameters()

post_fit_W_matrices = []
samples_after_fit = []

for i in range(n_samples):
    sampled_W = fitted_ergm.sample_network(sampling_method="NaiveMetropolisHastings", steps=1000)
    G = connectivity_matrix_to_G(sampled_W, directed=False)
    edge_count = len(G.edges())
    triangle_count = sum(nx.triangles(G).values()) // 3

    samples_after_fit.append({"edge_count": edge_count, "triangle_count": triangle_count})
    post_fit_W_matrices.append(sampled_W)

samples_after_fit_df = pd.DataFrame(samples_after_fit)
print("")

print(f"Mean of samples after fitting, calculated on {n_samples} samples- ")
print(samples_after_fit_df.mean())



Post fit ERGM parameters:
Number of nodes: 5
Thetas: [-0.15381744]
Normalization factor approx: 135.44816303424523
Is directed: False

Mean of samples after fitting, calculated on 10 samples- 
edge_count        8.4
triangle_count    6.0
dtype: float64


$$
\hat{\theta} = -0.1538 , \theta = -0.3369
\\
$$

edge count and triangle_count not looking better than the random ergm...